### 얼굴 필터 적용

In [4]:
import cv2
import mediapipe as mp
import numpy as np

# 람쥐 필터 이미지 (투명 PNG)
filter_img = cv2.imread('squirrel.png', cv2.IMREAD_UNCHANGED)

# mediapipe 얼굴 검출 초기화
mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils
face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# 필터 오버레이 함수
def overlay_filter(frame, filter_img, x, y, w, h):
    filter_resized = cv2.resize(filter_img, (w, h))

    for c in range(3):  # BGR 채널
        frame[y:y+h, x:x+w, c] = frame[y:y+h, x:x+w, c] * (1 - filter_resized[:, :, 3]/255.0) + \
                                 filter_resized[:, :, c] * (filter_resized[:, :, 3]/255.0)

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 좌우 반전
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape

    # 얼굴 감지
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            x = int(bboxC.xmin * w)
            y = int(bboxC.ymin * h)
            box_w = int(bboxC.width * w)
            box_h = int(bboxC.height * h)

            # 얼굴 기준으로 필터 위치 조정 (이마 쪽으로 위로 이동)
            offset_y = int(box_h * 0.4)
            x1 = max(0, x - int(box_w * 0.2))
            y1 = max(0, y - offset_y)
            x2 = min(w, x1 + int(box_w * 1.4))
            y2 = min(h, y1 + int(box_h * 1.5))

            filter_w = x2 - x1
            filter_h = y2 - y1

            if filter_w > 0 and filter_h > 0:
                overlay_filter(frame, filter_img, x1, y1, filter_w, filter_h)

    cv2.imshow('Squirrel Filter 🐿️', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 